<a href="https://colab.research.google.com/github/gueduar/trilha-python-dio-santander/blob/main/desafio_pipeline_python_datagui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pipeline de ETL Python**

## **E**tract
Extração dos dados sobre o plantio de árvores numa determinada localidade e dos dados referentes ao nomes científicos de algumas espécies além de informações sobre elas.

In [99]:
import pandas as pd

df_plantio = pd.read_excel('/content/bd_controle_plantas.xlsx')
df_especies = pd.read_csv('/content/lista_especies.csv',";")

df_plantio.rename(columns={'especie': 'NOME POPULAR'}, inplace = True)

s_arvores = df_plantio['NOME POPULAR']
s_arvores.dropna(inplace = True)

lista_nome_pop = list(set(s_arvores.tolist()))

<ipython-input-99-9bf1f86c1e26>:4: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df_especies = pd.read_csv('/content/lista_especies.csv',";")


## **T**ransform
Identifica e filtra apenas as plantas em que foi possível a identificação do nome científico

In [100]:
df = pd.DataFrame()

for nomepopular in lista_nome_pop:
  if nomepopular in df_especies['NOME POPULAR'].to_list():
    if df.empty == True:
      df = df_especies.loc[(df_especies['NOME POPULAR'].str.replace('-',' ') == nomepopular),['NOME POPULAR','NOME CIENTÍFICO']]
    else:
      df = pd.concat([df,pd.DataFrame(df_especies.loc[(df_especies['NOME POPULAR'] == nomepopular),['NOME POPULAR','NOME CIENTÍFICO']])])


Identifica se a planta está viva ou não após a criação do DataFrame que será extraído

In [101]:
from pandas.core.arrays.datetimelike import NaT
import numpy as np

df_final = pd.merge(df, df_plantio, on = "NOME POPULAR")

df_final = df_final[['ID','NOME POPULAR', 'NOME CIENTÍFICO', 'dt_plantio', 'dt_morte']].set_index('ID')

df_final['FL_PLANTA_VIVA'] = np.where(pd.isnull(df_final['dt_morte']) == True, True, False)

## **L**oad
Carregar DataFrame final para um arquivo CSV no Gooogle Drive

In [102]:
caminho_arquivo = '/content/drive/MyDrive/minhas_arvores.csv'

df_final.to_csv(caminho_arquivo)